In [43]:
%pip install transformers datasets torch
%pip install hf_xet
%pip install transformers[torch]
%pip install accelerate
%pip install --upgrade transformers accelerate
%pip install --upgrade torch datasets
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, average="binary"),
        "recall": recall_score(labels, preds, average="binary"),
        "f1": f1_score(labels, preds, average="binary"),
    }

In [ ]:
from datasets import ClassLabel

dataset_path = os.path.abspath("../data/synthesized_twitch_chats.csv")
dataset = load_dataset("csv", data_files=dataset_path)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

dataset = dataset.cast_column("labels", ClassLabel(names=["not_offensive", "offensive"]))

dataset = dataset["train"].train_test_split(test_size=0.2, seed=42, stratify_by_column="labels")

def tokenize(batch):
    return tokenizer(batch["message"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

In [ ]:
if os.path.exists("./saved_model/model"):
    model = BertForSequenceClassification.from_pretrained("./saved_model/model")
    tokenizer = BertTokenizer.from_pretrained("./saved_model/tokenizer")
else:
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-8,
        per_device_train_batch_size=16,
        num_train_epochs=2,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.save_model("./saved_model/model")
    tokenizer.save_pretrained("./saved_model/tokenizer")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\DELL\AppData\Local\Temp\ipykernel_11724\2246896089.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\DELL\AppData\Local\Temp\ipykernel_11724\2246896089.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\MyFolder02\Year3-1\HCI\Project\bert_venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\MyFolder02\Year3

Step,Training Loss


In [48]:
trainer.evaluate()

c:\MyFolder02\Year3-1\HCI\Project\bert_venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


c:\MyFolder02\Year3-1\HCI\Project\bert_venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


{'eval_loss': 0.7060182094573975,
 'eval_accuracy': 0.5070422535211268,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 1.3033,
 'eval_samples_per_second': 54.478,
 'eval_steps_per_second': 6.906,
 'epoch': 2.0}

In [63]:
text = "Go back to your country!"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
pred = outputs.logits.argmax(dim=1).item()
print("Offensive" if pred == 1 else "Not Offensive")

Not Offensive
